In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("Hitters.csv")
df = df.copy()
df.head()
#veri setimizde index problemi yok 


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [2]:
df.shape


(322, 20)

In [3]:
df.isnull().sum()
# salary değişkenlerimizde boş gözlemler bulunmaktadır 
# bunları ortalama ile dolduralım 


AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [4]:
df["Salary"].fillna(df.Salary.mean() , inplace = True)
# boş maaş değerlerini ortalama ile doldurduk aslında bunları ilerde tahmin ile dolduracağız

In [5]:
df.isnull().sum().any()
# datamızda boş değer var mı ? 


False

In [6]:
df.info()
# veri setimizdeki kategorik değişkenleri düzenliyelim 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
AtBat        322 non-null int64
Hits         322 non-null int64
HmRun        322 non-null int64
Runs         322 non-null int64
RBI          322 non-null int64
Walks        322 non-null int64
Years        322 non-null int64
CAtBat       322 non-null int64
CHits        322 non-null int64
CHmRun       322 non-null int64
CRuns        322 non-null int64
CRBI         322 non-null int64
CWalks       322 non-null int64
League       322 non-null object
Division     322 non-null object
PutOuts      322 non-null int64
Assists      322 non-null int64
Errors       322 non-null int64
Salary       322 non-null float64
NewLeague    322 non-null object
dtypes: float64(1), int64(16), object(3)
memory usage: 50.4+ KB


# Kategorik değişkenleri modellemeye hazır hale getirme 

In [7]:
from sklearn.preprocessing import LabelEncoder 
cat_list = ["NewLeague" , "Division" , "League"] 

lbe = LabelEncoder()
for i in cat_list:
    df[i] = lbe.fit_transform(df[i])

In [8]:
df.head(3)
# tip dönüşümlerini başarılı bir şekilde uyguladık 

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,0,0,446,33,20,535.925882,0
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,1,1,632,43,10,475.000000,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,0,1,880,82,14,480.000000,0


# Bagimli ve bağımsız değişken ayrışımı 

In [9]:
X = df.drop("Salary"  , axis = 1) # bagımsız degisken 

In [10]:
y = df["Salary"] #bagimli değisken 

# Veri setini Train_Test olarak ayırma

In [11]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.20)

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
# doğru parcalamışız

(257, 19)
(257,)
(65, 19)
(65,)


# Model Kurma


In [17]:
from sklearn.ensemble import BaggingRegressor

In [33]:
bag_model = BaggingRegressor(bootstrap_features = True)

In [34]:
bag_model.fit(X_train , y_train)

BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=True,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=None, verbose=0,
                 warm_start=False)

- Bootstrap_features = Ağaçların özelliklerini görmek için açacağız
- n_estimators = Ağaç sayısı

In [35]:
bag_model.n_estimators
# ağaç sayısı

10

In [36]:
bag_model.estimators_
# agaçların özlllikleri

[DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort=False, random_state=493344337, splitter='best'),
 DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort=False, random_state=1310450778, splitter='best'),
 DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_w

In [37]:
bag_model.estimators_[1]
# 1.ağacın özellikleri 

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1310450778, splitter='best')

In [38]:
bag_model.estimators_samples_
# her bir ağaçta kaç örnek var , index ve indeşleri

[array([245,  41,  46,   5, 184,  41,  87, 134, 256, 224,  54,  69, 184,
         51,  48, 222, 154, 251, 198, 100, 222,  37, 113,  89,  55,  48,
         54, 138, 189,   6, 122, 148,  63, 197, 101, 105, 143,  68, 246,
          4,  31, 220,  87,  36, 136, 215, 160, 173, 100,  18, 231,  71,
         79, 186, 124, 203, 121,  26, 141, 121,  11, 184,  60,  76, 224,
        155, 137,  32, 166,  11, 114,  45, 198,  71, 175, 182, 180,  83,
        149,  83, 237, 124, 121, 225, 187,  22, 204, 175, 239,   1,  55,
        143,  92, 119,   1, 154, 113, 147,  33, 101, 248,  20, 205,  94,
         48,  98, 188, 238, 222, 217, 147,  26, 107, 230, 156, 252, 169,
         58, 240, 198, 173,  81, 102, 103,  61,  32, 159,  57,  93, 215,
         83,  59, 164, 232,  53, 220, 139, 195, 145, 233, 177, 249, 181,
        224, 179, 209, 169, 186,  16,  99, 207, 200,  25, 157, 132,  56,
        221, 238, 145, 114, 171, 190, 216,  98, 115, 132,  56,  83, 180,
        106, 153, 102,  32, 143, 241, 140,  27,   8

In [39]:
bag_model.estimators_features_
# ağaçların kullandığı değişkenler 

[array([12,  1,  2, 15,  4,  0,  3,  9, 16,  4, 11,  5,  7, 14, 15, 14,  4,
        11,  6]),
 array([15,  3, 17,  8, 10,  9, 18, 12,  2,  9,  7, 14, 11,  4,  5, 10, 12,
         5,  1]),
 array([ 7,  8, 12, 11,  8, 10, 16, 10, 13,  4,  4,  9,  1, 12,  7,  4, 14,
        10, 14]),
 array([ 6,  2, 14, 18,  6,  0, 17, 10,  1,  4, 18,  5, 11,  7,  7, 10,  1,
        13,  6]),
 array([11, 12,  6,  8,  9,  6, 13, 14,  5, 17,  8,  3,  4, 17, 13,  3, 16,
        14,  7]),
 array([ 6,  8, 11,  1,  6,  8,  9, 13,  7,  8,  0,  5, 16,  6, 15, 16,  6,
        10, 14]),
 array([ 1,  7, 13, 10,  1,  7,  5,  1, 17, 14, 17,  1,  0,  5,  6,  2, 13,
         3,  7]),
 array([12,  0,  2,  5,  1,  3,  1, 15,  2,  3,  6, 18, 12,  3,  5,  7, 17,
         8,  8]),
 array([ 9, 12,  0, 14, 12, 12, 15,  2, 18,  1, 12,  7,  7, 13, 16,  0, 10,
         0, 16]),
 array([ 7,  3, 10,  4,  2, 13,  5, 16,  6, 10,  8, 10,  4,  1, 17,  3,  6,
         4,  4])]

## Tahmin


In [43]:
from sklearn.metrics import mean_squared_error

In [44]:
y_pred = bag_model.predict( X_test )

In [45]:
np.sqrt( mean_squared_error(y_test ,y_pred) )

266.80872714386163

## Agaca özel tahmin 

In [50]:
y_pred_2 = bag_model.estimators_[3].predict(X_test)

In [51]:
np.sqrt( mean_squared_error(y_test ,y_pred_2 ) )

619.7445367895754

- Baya hatalı agaçmış 

# Tuning 

In [53]:
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV


In [56]:
bag = BaggingRegressor()
bag_params = {"n_estimators" : range(2 ,20)}
bag_cv = GridSearchCV(bag ,
                     bag_params,
                     cv = 10 ,
                     n_jobs = -1 , 
                     verbose = 2).fit(X_train ,y_train)


Fitting 10 folds for each of 18 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    4.1s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [57]:
bag_cv.best_params_

{'n_estimators': 17}

# Final modeli 

In [59]:
from sklearn.ensemble import BaggingRegressor


In [60]:
bag_model = BaggingRegressor(n_estimators = 17).fit(X_train , y_train)


In [61]:
y_predict = bag_model.predict(X_test)

In [62]:
np.sqrt( mean_squared_error( y_test , y_pred ) )

266.80872714386163